mixtape

In [ ]:
from pydub import AudioSegment
import os, random

def smooth_fade_mixtape(folder, output="output/mixtape3.mp3", transition_ms=6000):
    files = [f for f in os.listdir(folder) if f.endswith((".mp3", ".wav"))]
    random.shuffle(files)  # shuffle for variety
    
    mixtape = None
    for i, file in enumerate(files):
        song = AudioSegment.from_file(os.path.join(folder, file))
        song = song.set_channels(2).set_frame_rate(44100)
        
        if mixtape is None:
            mixtape = song
        else:
            # define overlap region
            overlap = min(transition_ms, len(song), len(mixtape))
            
            # outgoing: fade out + high-pass to remove bass
            outgoing = mixtape[-overlap:].fade_out(overlap).low_pass_filter(4000)
            
            # incoming: fade in + low-pass to smooth highs
            incoming = song[:overlap].fade_in(overlap).low_pass_filter(4000)
            
            # overlay transition
            transition = outgoing.overlay(incoming)
            
            mixtape = mixtape[:-overlap] + transition + song[overlap:]
    
    mixtape.export(output, format="mp3")
    print(f"✅ Smooth fade mixtape generated: {output}")

# Example usage
smooth_fade_mixtape("mixtape")


video

In [ ]:
import subprocess
import os
from PIL import Image

def make_video_from_audio_optimized(image_path, audio_path, output_path="output/mixtape_vid3.mp4",
                                    video_resolution=(1280, 720), fps=1, preset="ultrafast"):
    """
    Merge a static image with audio to create a video.
    Optimized for long audios and high-resolution images.
    - Resizes image to desired resolution (even dimensions for libx264)
    - Uses low fps (1 fps) for static image to reduce processing
    - Uses ultrafast preset for speed
    - Re-encodes audio to AAC for MP4
    """
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image not found: {image_path}")
    if not os.path.exists(audio_path):
        raise FileNotFoundError(f"Audio not found: {audio_path}")

    # Ensure output folder exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Resize image if necessary
    img_resized_path = "temp_resized_image.jpg"
    img = Image.open(image_path)
    img = img.resize(video_resolution)
    img.save(img_resized_path)

    # FFmpeg command
    cmd = [
        "ffmpeg",
        "-y",
        "-loop", "1",
        "-i", img_resized_path,
        "-i", audio_path,
        "-c:v", "libx264",
        "-preset", preset,
        "-tune", "stillimage",
        "-r", str(fps),  # low fps for static image
        "-c:a", "aac",
        "-b:a", "192k",
        "-shortest",
        output_path
    ]

    try:
        subprocess.run(cmd, check=True)
        print(f"✅ Video created: {output_path}")
    except subprocess.CalledProcessError as e:
        print("❌ FFmpeg failed! Error:", e)
    finally:
        # Clean up temporary resized image
        if os.path.exists(img_resized_path):
            os.remove(img_resized_path)

# Example usage
image_path = r"images/hh.jpg"
audio_path = r"output/mixtape3.mp3"
make_video_from_audio_optimized(image_path, audio_path)


description

In [ ]:
import os
from pydub import AudioSegment

def generate_youtube_description_with_timestamps(track_folder, mixtape_name="Afro House Mixtape", genre="Afro House", start_time_sec=0):
    """
    Generates a YouTube description with tracklist, timestamps, and hashtags.
    Assumes audio files are in order of playback.
    """
    audio_extensions = (".mp3", ".wav", ".flac", ".aac", ".ogg", ".m4a")
    tracks = [f for f in os.listdir(track_folder) if f.lower().endswith(audio_extensions)]
    if not tracks:
        return "No audio tracks found in the folder!"

    tracks.sort()  # optional: sort alphabetically

    description = f"🔥 {mixtape_name} 🔥\n"
    description += f"Genre: {genre}\n\n"
    description += "🎵 Tracklist:\n"

    current_time = start_time_sec

    for idx, track in enumerate(tracks, 1):
        track_path = os.path.join(track_folder, track)
        audio = AudioSegment.from_file(track_path)
        duration_sec = len(audio) // 1000  # duration in seconds

        # Format timestamp mm:ss
        minutes = current_time // 60
        seconds = current_time % 60
        timestamp = f"{minutes:02d}:{seconds:02d}"

        track_name = os.path.splitext(track)[0]
        description += f"{timestamp} - {track_name}\n"

        current_time += duration_sec  # increment for next track

    description += "\n💽 Download/Listen links:\n"
    description += "You can find these tracks online or on Spotify/Apple Music.\n\n"
    description += "🎧 Follow for more Afro House mixes!\n\n"

    hashtags = [
        "#AfroHouse", "#HouseMusic", "#EDM", "#DanceMusic", "#Mixtape",
        "#ElectronicMusic", "#DJMix", "#PartyMusic", "#DeepHouse", "#AfroHouseVibes"
    ]
    description += " ".join(hashtags)

    return description

# Example usage
track_folder = r"mixtape"
description_text = generate_youtube_description_with_timestamps(
    track_folder,
    mixtape_name="Afro House Summer Mix",
    genre="Afro House"
)

print(description_text)

# Optional: save to file
with open("youtube_description.txt", "w", encoding="utf-8") as f:
    f.write(description_text)
